# Dataset-2: Bank-Marketing

In [53]:
import numpy as np 
import pandas as pd 
import math
import operator
from sklearn.model_selection import train_test_split

---
## Section-1: Neural Network Class

The same neural network as the one described in the other notebook will be used to analyze this dataset. The implementation of neural network class is copied as is below.

In [54]:
class Neural_Network_Imp:
    
    # defining constructor and initializing values
    def __init__(self,X, y, X_test, y_test, input_learning_rate, input_epochs_count):      
                
        # initializing the data
        
        self.X_test_data = X_test
        self.y_test_data = y_test
        self.y_data = y[:,None]
        self.X_data = X
        
        # calcultaing the number of hidden nodes
        # with n input and m output neurons, the hidden layer would have sqrt(n ∗ m) neurons.
        # reference:
        # [1] Masters, Timothy. Practical neural network recipes in C++. Morgan Kaufmann, 1993.
        # [2] http://www.iitbhu.ac.in/faculty/min/rajesh-rai/NMEICT-Slope/lecture/c14/l1.html
        
        number_hidden_nodes= int(round(math.sqrt(len(X[0])*self.y_data.shape[1]) )) 
        # print("Calculated number of hidden nodes-->",number_hidden_nodes )
        # since we have a binary classification so the value of m i.e., output neurons will be 1.
        
        # setting the value for number_input_nodes, number_output_nodes, input_learning_rate and input_epochs_count
        
        self.number_input_nodes = len(X[0])
        self.number_hidden_nodes = number_hidden_nodes
        self.number_output_nodes = self.y_data.shape[1]
        self.input_learning_rate = input_learning_rate
        
        print("Model Details:")
        print("1. Number of input nodes-->",self.number_input_nodes )
        print("2. Calculated number of hidden nodes by formula sqrt(input nodes ∗ output nodes)-->",number_hidden_nodes )
        print("3. Number of output nodes-->",self.number_output_nodes )
        print("4. Learning rate for the model inputted by the user  -->",self.input_learning_rate )
        print("5. Epochs for the model inputted by the user  -->",input_epochs_count )
        print("\n")
        
        # Initilaizing random weights between input layer --> hidden layer and from hidden layer --> output layer
        
        np.random.seed(4)
        # weight matrix from input to hidden layer    
        self.weightInputHidden = np.random.random((self.number_input_nodes, self.number_hidden_nodes)) 
        # weight matrix from hidden to output layer
        self.weightHiddenOutput = np.random.random((self.number_hidden_nodes, self.number_output_nodes)) 
        
       #self.train(epochs)
        self.functionForTrainDataSet(input_epochs_count)
        self.functionForTestingDataSet()
        
    
    # function for Activation function: Sigmoid
    
    def functionForCalcultingAFSigmoid(self, input_data):
        return (1/(1+np.exp(-input_data)))
        
    # function for derivative of Sigmoid Activation function
    
    def functionForCalcultingAFSigmoidDerivative(self, input_data):
        return input_data * (1 - input_data)
        
    # function to train our dataset and build a model
    
    def functionForTrainDataSet(self, input_epochs_count):
        
        for counter in range(input_epochs_count):
            
            #forward propagation through our network
            #product of input and hidden layers neurons
            self.temp = (self.X_data[:,:,None]*self.weightInputHidden[None,:,:]).sum(axis=1)            
            
            inputHiddenLayer = self.functionForCalcultingAFSigmoid(self.temp)

            #product of hidden and output layers neurons
            self.temp2 = (inputHiddenLayer[:,:,None]*self.weightHiddenOutput[None,:,:]).sum(axis=1)            
            hiddenOutputLayer = self.functionForCalcultingAFSigmoid(self.temp2)
        
            # backward propagate through the network
            
            #calculating error in the output
            #errorInPrediction is calculated by subtracting predicted output from the original output
            errorInPrediction = self.y_data-hiddenOutputLayer
            
            #calculate how far off each layer is
            hiddenOutputLayer_difference = errorInPrediction * self.functionForCalcultingAFSigmoidDerivative(hiddenOutputLayer)
            inputHiddenLayer_difference = (hiddenOutputLayer_difference[:,:,None]*self.weightHiddenOutput.T[None,:,:]).sum(axis=1) * self.functionForCalcultingAFSigmoidDerivative(inputHiddenLayer)
            
            # from the error values found, propagating backward and adjusting the error so that predicting result becomes more accurate
            self.weightHiddenOutput = np.add(self.weightHiddenOutput,(inputHiddenLayer.T[:,:,None]*hiddenOutputLayer_difference[None,:,:]).sum(axis=1)  * self.input_learning_rate)          
            self.weightInputHidden = np.add(self.weightInputHidden, (self.X_data.T[:,:,None]*inputHiddenLayer_difference[None,:,:]).sum(axis=1) * self.input_learning_rate)
        
        print("Output Details:")
        print('The % of error is-->', (abs(errorInPrediction)).mean())
        
    # function to test our built model and to check the accuracy with the test dataset    
        
    def functionForTestingDataSet(self):
        
        inputHiddenLayerTest = self.functionForCalcultingAFSigmoid((self.X_test_data[:,:,None]*self.weightInputHidden[None,:,:]).sum(axis=1))
        hiddenOutputLayerTest = self.functionForCalcultingAFSigmoid((inputHiddenLayerTest[:,:,None]*self.weightHiddenOutput[None,:,:]).sum(axis=1))
        
        listPredictedOutput = []
        numberOfCorrectPredict = 0    
        
        for count in range(len(hiddenOutputLayerTest)):
            if hiddenOutputLayerTest[count] >= 0.5:
                tempPrediction = 1
            else:
                tempPrediction = 0

            if tempPrediction == self.y_test_data[count]:
                numberOfCorrectPredict += 1
                
            listPredictedOutput.append(tempPrediction)

        print "Accuracy of our model with Test data is--> ", ((float(numberOfCorrectPredict)/len(y_test))*100),'%'

---
## Section-2: Data Wrangling
In this section, we load the dataset into pandas. We grok the dataset, clean it as we see fit and divide it into featues and labels column.

In [55]:
df = pd.read_csv ('../data/bank-full.csv', sep = ';')
df.head ()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [56]:
df.describe ()

,age,balance,day,duration,campaign,pdays,previous
count,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000
mean,40.936210,1362.272058,15.806419,258.163080,2.763841,40.197828,0.580323
std,10.618762,3044.765829,8.322476,257.527812,3.098021,100.128746,2.303441
min,18.000000,-8019.000000,1.000000,0.000000,1.000000,-1.000000,0.000000
25%,33.000000,72.000000,8.000000,103.000000,1.000000,-1.000000,0.000000
50%,39.000000,448.000000,16.000000,180.000000,2.000000,-1.000000,0.000000
75%,48.000000,1428.000000,21.000000,319.000000,3.000000,-1.000000,0.000000
max,95.000000,102127.000000,31.000000,4918.000000,63.000000,871.000000,275.000000


The above description of the dataset shows only the numeric columns. In the next step, we clean the dataset to keep only these values and the label column.

### Step-2.1: Data Cleaning
The non-numeric columns cannot be handled by our neural network as-is. Factorizing these values has the potential drawback of over-training the network; when used alongside numeric columns which are more appropriate for our neural network. For this reason, we are dropping the non-numeric columns from our dataset in this step.

Even without the non-numeric columns, we have enough (numeric) features in our dataset to reasonably train our neural network. We also **normalize** the numeric features to aid in the training process.

In [57]:
df2 = df [['age', 'balance', 'day', 'duration', 'campaign', 'pdays']].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
df2 ['y'] = df ['y']
df2.head ()

,age,balance,day,duration,campaign,pdays,y
0,0.519481,0.092259,0.133333,0.053070,0.0,0.0,no
1,0.337662,0.073067,0.133333,0.030704,0.0,0.0,no
2,0.194805,0.072822,0.133333,0.015453,0.0,0.0,no
3,0.376623,0.086476,0.133333,0.018707,0.0,0.0,no
4,0.194805,0.072812,0.133333,0.040260,0.0,0.0,no


### Step-2.2: Separate Dataset Columns into Features and Labels
For this dataset, we are using the final column **'y'** as label which indicates whether a customer accepted the bank's offer or not. The remaining numeric columns from the datasets are used as features.

In [58]:
X = df2.iloc[:, 0:5].values
y = df2.iloc[:, 6].values
y = pd.factorize(y)[0]
X, y

(array([[ 0.51948052,  0.09225936,  0.13333333,  0.05307035,  0.        ],
        [ 0.33766234,  0.07306666,  0.13333333,  0.03070354,  0.        ],
        [ 0.19480519,  0.07282153,  0.13333333,  0.01545344,  0.        ],
        ..., 
        [ 0.7012987 ,  0.12468905,  0.53333333,  0.22915819,  0.06451613],
        [ 0.50649351,  0.07886805,  0.53333333,  0.10329402,  0.0483871 ],
        [ 0.24675325,  0.09977666,  0.53333333,  0.07340382,  0.01612903]]),
 array([0, 0, 0, ..., 1, 0, 0]))

---
## Section-3: Classification
In this section, we train and test our neural network for the dataset at hand.

### Step-3.1: Split Data into Training / Test Sets

In [59]:
# Splitting training and testing in 80:20 ratio
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Step-3.2: Neural Network Classification

In [60]:
nniInstance = Neural_Network_Imp(X_train, y_train, X_test, y_test, 0.1, 5000)

Model Details:
('1. Number of input nodes-->', 5)
('2. Calculated number of hidden nodes by formula sqrt(input nodes \xe2\x88\x97 output nodes)-->', 2)
('3. Number of output nodes-->', 1)
('4. Learning rate for the model inputted by the user  -->', 0.1)
('5. Epochs for the model inputted by the user  -->', 5000)


Output Details:
('The % of error is-->', 0.11684361866843619)
Accuracy of our model with Test data is-->  88.245051421 %


---
## Conclusion

The neural network seems to be doing an adequate job. We can also immediately realize the **utility of Neural Network approach** in providing a generic algorithm for classification since the network used for detecting credit card fraud is applicable as is for classifying bank marketing data.